In [36]:
import warnings
warnings.filterwarnings('ignore')


import os 
import yaml
from crewai import Agent, Task, Crew, LLM

In [37]:
llm = LLM(model = "ollama/qwen2:1.5b", base_url = "http://localhost:11434")

In [38]:
#from langchain_ollama import OllamaLLM
# model = OllamaLLM(model="qwen2:1.5b")
# result = model.invoke(input = "hello world")
# print(result)
# #llm = OllamaLLM(model = "ollama/qwen2:1.5b", base_url = "http://localhost:11434")

* Agents                                
    * Project Planner
    * Estimation Analyst
    * Allocation Strategist

* Tasks
    * Task Breakdown
    * Time Estimation
    * Resource Allocation

**Project Plan**
* Taks / Allocation
* Milestones

# Tasks and Agents YAML files

In [39]:
files = {
    'agents' : 'config/agents.yaml',
    'tasks'  : 'config/tasks.yaml'
}

#Load Configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agent_config = configs['agents']
task_config  = configs['tasks']

## Create Pydantic Models for Structured Output

In [40]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimator(BaseModel):
    task_name:str =                 Field(..., descripton= "name of the task")
    estimated_time_hourse:float=    Field(..., descripton= "Estimated time to complete  the task i hours")
    required_resources:List[str]=   Field(..., descripton= "List of resouces required to complete the task")
class Milestone(BaseModel):
    milestone_name:str=             Field(..., description= "Name of the milestone")
    tasks:List[str]=                Field(..., description= "List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks:List[TaskEstimator]=      Field(..., description="List of tasks with their estimates")
    milestone:List[Milestone]=      Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [41]:
# Create AGents
project_planning_agent = Agent(config = agent_config['project_planning_agent'], llm = llm)

estimation_agent = Agent(config = agent_config['estimation_agent'], llm= llm )

resource_allocation_agent = Agent(config= agent_config['resource_allocation_agent'], llm = llm)

# Creating Tasks
task_breakdown = Task(config=task_config['task_breakdown'], agent= project_planning_agent)

time_resource_estimtation = Task(config = task_config['time_resource_estimation'], agent = estimation_agent)

resource_allocation = Task(config=task_config['resource_allocation'], agent=resource_allocation_agent, output_pydantic= ProjectPlan)

crew = Crew( agents=[project_planning_agent, estimation_agent, resource_allocation_agent],
             tasks=[task_breakdown, time_resource_estimtation, resource_allocation], verbose = True)

In [42]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off crew

In [43]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru